# Dataset Analysis


In [ ]:
# PyTorch
import torch
from torch import nn

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

## Import the Fashion MNIST dataset

This guide uses the [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) dataset which contains 70,000 grayscale images in 10 categories. The images show individual articles of clothing at low resolution (28 by 28 pixels), as seen here:

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Figure 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST samples</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

Fashion MNIST is intended as a drop-in replacement for the classic [MNIST](http://yann.lecun.com/exdb/mnist/) dataset—often used as the "Hello, World" of machine learning programs for computer vision. The MNIST dataset contains images of handwritten digits (0, 1, 2, etc.) in a format identical to that of the articles of clothing you'll use here.

This guide uses Fashion MNIST for variety, and because it's a slightly more challenging problem than regular MNIST. Both datasets are relatively small and are used to verify that an algorithm works as expected. They're good starting points to test and debug code.

Here, 60,000 images are used to train the network and 10,000 images to evaluate how accurately the network learned to classify images. You can access the Fashion MNIST directly from PyTorch through *torchvision* library.

In [ ]:
# Loading the Fashion-MNIST dataset
from torchvision import datasets, transforms


# Download and load the training data

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.
])

# datasets
trainset = datasets.FashionMNIST('./data',
    download=True,
    train=True,
    transform=transform)

testset = datasets.FashionMNIST('./data',
    download=True,
    train=False,
    transform=transform)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# A PyTorch implementation


## Import and wrap the Fashion MNIST dataset

A good practice when using PyTorch is to extend the *Dataset* class to better manage data. Every subclasses must extend the following methods:

*   **\_\_init\_\_** : class constructor, here are specified the data and the parameters;
*   **\_\_getitem\_\_**: method called implicit later by the *dataloader* to pass the data to the model;
*   **\_\_len\_\_**: method that specify the length of the dataset;

Other methods can be added to better manage the data.


In [ ]:
from torch.utils.data import Dataset

class FashionDataset(Dataset):


In [ ]:
dataset_train = FashionDataset(trainset)
dataset_test = FashionDataset(testset)

## Build the model

Following the PyTorch best practice, a model is built by extending the `nn.Module` class. Even in this case every subclasses have to extend the following methods:

*   **\_\_init\_\_**: class constructor, here are specified the layers of the model;
*   **forward**: method that rapresent the forward pass of the model

Methods for wrapping training, evaluation and prediction operation can be added in this class.



In [ ]:
from tqdm import tqdm

class FashionModel(nn.Module):

    def __init__(self):
        super(FashionModel, self).__init__()


    def forward(self, x):

        return x


    def train_classifier(self, dataloader, epochs, criterion, optimizer, device):
        train_loss = []
        train_accs = []

        for ep in range(epochs):
            self.train()
            running_loss = 0.0
            acc = 0
            total = 0

            for it, (images, labels) in enumerate(tqdm(dataloader)):

                images = images.to(device)
                labels = labels.to(device)

                # Forward pass
                logits = self.forward(images)
                loss = criterion(logits, labels)

                running_loss += loss.item()

                # Backpropagation
                loss.backward()
                optimizer.step()

                optimizer.zero_grad()

                # Accuracy
                predictions = torch.max(logits, 1).indices.to(device)

                acc += (predictions.detach().cpu().numpy() == labels.detach().cpu().numpy()).sum()
                total += len(labels)

            epoch_loss = running_loss/len(train_loader)
            train_loss.append(epoch_loss)

            epoch_acc = acc*100/total
            train_accs.append(epoch_acc)

            print(f"Epoch {ep+1}: Loss {round(epoch_loss, 3)} - Accuracy {round(epoch_acc, 2)}%\n")

        return train_loss, train_accs


    def eval_classifier(self, dataloader, device):
        self.eval()
        val_acc = 0

        for it, (images, labels) in enumerate(tqdm(dataloader)):

            with torch.no_grad():
                images = images.to(device)

                # Forward pass
                logits = self.forward(images)

                # Accuracy
                predictions = torch.max(logits, 1).indices

                val_acc += (predictions.detach().cpu().numpy() == labels.detach().cpu().numpy()).sum()

        print(f"\nTrained model Accuracy on testset: {round(val_acc*100/len(dataloader), 3)}%")


    def predict(self, dataloader, device):
        self.eval()
        softmax = nn.Softmax(dim=1)

        predictions = []
        for it, (images, labels) in enumerate(tqdm(dataloader)):

            with torch.no_grad():
                images = images.to(device)

                logits = self.forward(images)
                preds = softmax(logits)

                predictions.append(preds.detach().cpu().numpy()[0])

        return predictions

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = FashionModel().to(device)

print(model)

In [ ]:
from torch import optim

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss().to(device)

batch_size = 64

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset_test, batch_size=1)

epochs = 10

In [ ]:
train_loss, train_accs = model.train_classifier(train_loader, epochs, criterion, optimizer, device)

In [ ]:
plt.plot(range(epochs), train_loss)
plt.xlabel("No. of Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.show()

In [ ]:
plt.plot(range(epochs), train_accs)

plt.xlabel("No. of Epoch")
plt.ylabel("Accuracy %")
plt.title("Training Accuracy")
plt.show()

### Evaluate accuracy

Next, compare how the model performs on the test dataset:

In [ ]:
model.eval_classifier(test_loader, device)

### Make predictions

With the model trained, you can use it to make predictions about some images.
Attach a softmax layer to convert the model's linear outputs—[logits](https://developers.google.com/machine-learning/glossary#logits) to probabilities, which should be easier to interpret.

In [ ]:
predictions = model.predict(test_loader, device)

Graph this to look at the full set of 10 class predictions.

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

### Verify predictions

With the model trained, you can use it to make predictions about some images.

Let's look at the 0th image, predictions, and prediction array. Correct prediction labels are blue and incorrect prediction labels are red. The number gives the percentage (out of 100) for the predicted label.

In [ ]:
test_images = dataset_test.get_images()
test_labels = dataset_test.get_labels()

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()